In [1]:
# imports
import pandas as pd

In [2]:
# function takes dataframe and transforms into shares
def transform_share(df):
    columns = df.columns
    total = df.sum(axis=1)
    for i in columns:
        if i in ["Municipality", "Years", "Geographic Group"]:
            pass
        else:
            df[i] = df[i]/total
    
    return df

In [3]:
# transform education columns
def transform_edu(df, sex):
    df = df.rename(columns={'With no educational qualifications':'no_' + sex,
                          'Compulsory education 1st cycle':'first_' + sex,
                           'Compulsory education 2nd cycle': "second_" + sex,
                           'Compulsory education 3rd cycle': "third_" + sex,
                           'Upper-secondary': "upper_"+ sex,
                           'Middle level': "middle_"+ sex,
                           'Higher education': "higher_"+ sex
                          })
    return df

In [4]:
# transform type columns
def transform_type(df, sex):
    df = df.rename(columns={'Looking for 1st job':'first_job_' + sex,
                          'Looking for a new job':'new_job_' + sex,
                          })
    return df

In [5]:
# change name to Municipality
def transform_to_muni(df):
    df = df.rename(columns={'Years':'Municipality',
                           "Geographic Group":'Municipality'})
    df.set_index("Municipality", inplace=True)
    return df

In [6]:
# load all files into pandas df's
unemp = pd.read_excel("data_files/2011/unemp_rate_2011.xlsx")
all_age = pd.read_excel("data_files/2011/unemployment_age_2011.xlsx")
female_edu = pd.read_excel("data_files/2011/unemp_female_edu_2011.xlsx")
male_edu = pd.read_excel("data_files/2011/unemp_male_edu_2011.xlsx")
female_type = pd.read_excel("data_files/2011/unemp_female_type_2011.xlsx")
male_type = pd.read_excel("data_files/2011/unemp_male_type_2011.xlsx")

In [7]:
# prep unemp
unemp = transform_to_muni(unemp)
unemp = unemp.rename(columns={'Total':'Unemployment_Rate'})
unemp["Unemployment_Rate"] = unemp["Unemployment_Rate"]/100

In [8]:
# prep all_age
all_age = all_age.drop(["Total"], axis=1)
all_age = transform_to_muni(all_age)
all_age = transform_share(all_age)

In [9]:
# prep edu
female_edu = transform_to_muni(female_edu)
female_edu = transform_edu(female_edu, "female")
female_edu = transform_share(female_edu)

male_edu = male_edu.drop(["Geographic Group"], axis=1)
male_edu = transform_edu(male_edu, "male")
male_edu = transform_to_muni(male_edu)
male_edu = transform_share(male_edu)

In [10]:
# prep type
female_type = transform_to_muni(female_type)
female_type = transform_type(female_type, "female")
female_type = transform_share(female_type)

male_type = transform_to_muni(male_type)
male_type = transform_type(male_type, "male")
male_type = transform_share(male_type)

In [11]:
# get a list of all dfs
dfs = [all_age, female_edu, male_edu, female_type, male_type]

In [12]:
# concat all df's by looping over it
for d in dfs:
    unemp = unemp.join(d, on="Municipality")

In [32]:
unemp.head()

,Unemployment_Rate,15 - 24,25 - 34,35 - 44,45 - 54,55 - 64,65 or more,no_female,first_female,second_female,...,Skilled workers,Semi-skilled workers,Unskilled workers,Understudies and apprentices,Total,"Agriculture, farming of animals, hunting, agroforestry and fishing","Industry, construction, energy and water",Manufacturing,Construction,Services
Municipality,,,,,,,,,,,,,,,,,,,,,
Arcos de Valdevez,10.2,0.194757,0.242197,0.228464,0.239700,0.094881,0.000000,0.010101,0.148990,0.106061,...,712.5,633.3,606.2,589.7,764.5,664.9,753.5,777.4,649.1,777.3
Caminha,13.1,0.194503,0.263214,0.205074,0.214588,0.121564,0.001057,0.008097,0.089069,0.101215,...,762.0,644.6,577.6,576.2,854.9,1197.4,757.7,682.1,781.2,871.0
Melgaço,9.7,0.233216,0.307420,0.236749,0.183746,0.038869,0.000000,0.000000,0.095238,0.068027,...,752.7,571.3,551.9,577.0,753.1,590.6,734.0,732.2,686.0,764.9
Monção,9.8,0.201125,0.260197,0.229255,0.216596,0.092827,0.000000,0.018405,0.076687,0.098160,...,743.2,636.7,611.6,575.8,768.1,665.3,728.2,710.7,672.7,798.2
Paredes de Coura,11.1,0.211538,0.254808,0.245192,0.199519,0.088942,0.000000,0.000000,0.130208,0.098958,...,660.3,621.2,602.1,558.3,740.0,704.8,667.0,682.1,610.5,802.8


In [29]:
# get data for average earnings
wage_edu = pd.read_excel("data_files/2011/earnings_edu_2011.xlsx")
wage_quali = pd.read_excel("data_files/2011/earnings_quali_2011.xlsx")
wage_sector = pd.read_excel("data_files/2011/earnings_sector_2011.xlsx")

In [30]:
wage_edu = transform_to_muni(wage_edu)
wage_quali = transform_to_muni(wage_quali)
wage_sector = transform_to_muni(wage_sector)

In [31]:
# rename total to average_wage
wage_sector = wage_sector.rename(columns={'Total':'avg_wage'})

In [24]:
wage = [wage_edu, wage_quali, wage_sector]

In [25]:
# concat all df's by looping over it
for d in wage:
    unemp = unemp.join(d, on="Municipality")

In [14]:
unemp.to_pickle("./unemp.pkl")